In [8]:
## Generate counter_tb.sv using a yield-based python model of the counter
import itertools as it

def gate(sequence, enables):
    a = iter(sequence)
    b = next(a)
    for en in enables:
        yield b
        if en:
            b = next(a)

def counter_model(stimulus, **kwargs):
    reset_value = kwargs['reset_value'] if 'reset_value' in kwargs else 2
    increment = kwargs['increment'] if 'increment' in kwargs else 3
    max_value = kwargs['max_value'] if 'max_value' in kwargs else 10
    enable = stimulus()
    enable_tee = iter(it.tee(enable, 3))
    counter_sequence = gate(it.cycle(range(reset_value, max_value+1, increment)), next(enable_tee))
    counter_tee = iter(it.tee(counter_sequence, 2))
    carry_sequence = map(lambda x: x + increment > max_value, next(counter_tee))
    return zip(it.count(), next(enable_tee), next(counter_tee), carry_sequence)

def get_enables():
    ## Yield (reset, enable) tuples
    # start with a reset cycle
    yield False
    # wait ten cycles
    for _ in range(10):
        yield False
    # enable for twenty cycles
    for _ in range(20):
        yield True
    # disable and finish
    for _ in range(10):
        yield False

In [9]:
def write_golden(model, stimulus, **kwargs):
    width = kwargs['width'] if 'width' in kwargs else 4
    reset_value = kwargs['reset_value'] if 'reset_value' in kwargs else 0
    increment = kwargs['increment'] if 'increment' in kwargs else 1
    max_value = kwargs['max_value'] if 'max_value' in kwargs else 9
    with open('hdl/counter_tb.sv', 'w') as golden:
        golden.writelines(f"module counter_tb;\n")
        golden.writelines(f"    logic clk = 0;\n")
        golden.writelines(f"    always #5 clk = ~clk;\n")
        golden.writelines(f"    logic reset;\n")
        golden.writelines(f"    initial begin\n")
        golden.writelines(f"        reset = 1;\n")
        golden.writelines(f"        @(posedge clk) reset <= 0;\n")
        golden.writelines(f"    end\n")
        golden.writelines(f"    logic enable;\n")
        golden.writelines(f"    logic [{width-1}:0] count;\n")
        golden.writelines(f"    logic carry;\n")
        golden.writelines(f"    counter #(\n"
                          f"        .width({width}),\n"
                          f"        .reset_value({reset_value}),\n"
                          f"        .increment({increment}),\n"
                          f"        .max_value({max_value})\n"
                          f"    ) dut (.*);\n")
        result = model(stimulus, **kwargs)
        _, enable, count, carry = result.__next__()
        golden.writelines(f"    initial begin\n")
        golden.writelines(f"        enable = {1 if enable else 0};\n")
        last_enable = None
        for _, enable, count, carry in result:
            golden.writelines(f"        @(posedge clk);\n")
            if last_enable != enable:
                golden.writelines(f"        enable <= {1 if enable else 0};\n")
            last_enable = enable
            golden.writelines(f"        #1 assert (count == 'd{count} && carry == {1 if carry else 0}) else\n")
            golden.writelines(f"            $fatal(1, \"Something didn't match the expected result\");\n")
        golden.writelines(f"        @(posedge clk);\n")
        golden.writelines(f"        $display(\"Test passed\");\n")
        golden.writelines(f"        $finish;\n")
        golden.writelines(f"    end\n")
        golden.writelines(f"endmodule\n")

write_golden(counter_model, get_enables, width=4, reset_value=1, increment=3, max_value=15)
